In [ ]:
import matplotlib.pyplot as plt

from rick_tsp import *

Description:
This is the last block of code. It calls all of the above functions in order to solve a TSP, with arguments defined in-line. I still need to package this into a single pretty function, but rather than making it pretty, I have spent my time playing with the code and trying to learn more. I find it easier to leave this as an open block of code while still testing it.
      
Parameters would be: rand, num_towns, population_size, num_gen, mrate  
Returns: would not return anything, but would print out the results.

Rick Comments:
- I feel as though I have reached a point of deminishing returns with this project. Although there is still SO much room for improvement, I think that my time would be better spent applying my learning to a new project (I have another optimization problem in mind. It is a replenishment order optimizer for one of my clients, I have already built the tool with Solver in Excel, and I think it is very well suited to Python.)

In [ ]:
# Randomly create towns to visit:
num_towns=20
rand=1
Towns_To_Visit=create_towns(rand,num_towns)

# generate the To-From Matrix:
TFM=generate_TFM()

# Create a sample population of appropriate size:
population_size=6
population=generate_population(population_size)

# create a df to track the total distance of the fittest known route
fittest_known=pd.DataFrame(index=np.arange(1), columns=np.arange(1))

# Breed for the desired number of generations
num_gen=500
for i in range(num_gen):
    # Determine fitness of population:
    population_fitness=determine_fitness(population)
    # track fittest_known route
    fittest_known=fittest_known.append(population_fitness.nsmallest(1,0))
    fittest_known=fittest_known.reset_index(drop=True)

    # Select Parents to breed with
    # The parents are the two fittest individualts in the population
    parents_index=population_fitness.nsmallest(2,0).index
    parents=population.iloc[parents_index]
    # reset index of parents df... I still need to build in some logic to avoid
    # having to do this here(or at all...?)
    parents=parents.reset_index(drop=True)

    # Breed with the parents and mutate their offspring at a desired rate:
    mrate=1
    breed_and_mutate(mrate)
    # in case any of the children are identical to already-existing routes: drop duplicates
    population=population.drop_duplicates()
    population=population.reset_index(drop=True)


    # determine population fittenss for population WITH children
    population_fitness=determine_fitness(population)
    # now look at the population(with the children included), and get rid of
    # the two worst options (i.e. where fitness is longest)
    # this is to keep the size of the population constant
    winners_index=population_fitness.nsmallest(population_size,0).index
    population=population.iloc[winners_index]
    #reset index of parents df
    population=population.reset_index(drop=True)

    
# Present results in a nice way
print('\nYour resulting population is:') 
print(population)    
population_fitness=determine_fitness(population)
population_fitness.loc[0]='Route length'
print('\nThe route lengths associated with your population are:')
print(population_fitness)

print('\nThis plot shows you the distance of the best known route after a given number of itterations')
plt.plot(fittest_known)
plt.title('Distance of best candidate')
plt.ylabel('Distance of tour')
plt.xlabel('Number of itterations completed')
plt.show